In [1]:
from funcs import load_matrices, matrix_funcs
import numpy as np
import matplotlib.pyplot as plt
from scipy.special import airy, airye, riccati_jn, riccati_yn, iv, kv
import os


def riccati_in(n, x):
    f = np.array([np.sqrt(x)*iv(l+(1./2), x) for l in range(n+1)])
    fp = np.array([(x*(iv(l-(1./2), x)+iv(l+(3./2), x))+iv(l+(1./2), x))/(2*np.sqrt(x)) for l in range(n+1)])
    return f, fp


def riccati_kn(n, x):
    f = np.array([np.sqrt(x)*kv(l+(1./2), x) for l in range(n+1)])
    fp = np.array([(-x*(kv(l-(1./2), x)+kv(l+(3./2), x))+kv(l+(1./2), x))/(2*np.sqrt(x)) for l in range(n+1)])
    return f, fp


dirs = [n for n in os.listdir() if (os.path.isdir(n) and ((n[-2:]=='_p') or (n[-2:]=='_s')))]
# dirs = ['H_test_3_im']
for dir in dirs:
    r, hls, ddrls = load_matrices(dir)
    print(dir)
    emin = hls[-1,0,0]
    n = len(hls[0])
    shift = np.array([emin*np.eye(len(hls[0]), dtype=complex)]*len(r))
    vd = (hls-shift)/4.637

    hfunc, ddrfunc = matrix_funcs(r, vd, ddrls)
    emax = np.abs(np.array(hfunc(r[0])[-1,-1]))

    w = lambda rs, l, e: np.array([e*np.eye(n)-(hfunc(r)+l*(l*1)/(r**2)*np.eye(n)) for r in rs])
    wp = lambda rs, l: np.array([-(hfunc(r,1)-2*l*(l*1)/(r**3)*np.eye(n)) for r in rs])

    es = np.array([1])
    for i in range(n):
        emax = np.abs(np.array(hfunc(r[0])[i,i]))
        emin = np.abs(np.array(hfunc(r[-1])[i,i]))
        es = np.concatenate((es, np.power(10, np.arange(np.log10(emin if emin!=0 else 1), np.log10(emax), 0.01))))
    es = np.unique(np.round(np.sort(es)))
    sigmas = np.zeros((len(es), n, n))
    ecm = es*4.637+np.real(hls[-1,0,0])
    for ei in range(len(es)):
        e = es[ei]
        lmax = int(np.round(np.sqrt(e)*np.max(r[-1]))*0.8)
        ks = np.sqrt(np.abs(np.diag(e*np.eye(n)-hfunc(r[-1]))))
        yf = np.zeros((lmax, n, n), dtype=complex)
        nt = np.sum(np.diag(vd[-1]) < e)
        for l in range(lmax):
            wtest = w(r[:-1]+np.diff(r)/2, l, e)
            wptest = wp(r[:-1]+np.diff(r)/2, l)
            k2 = np.zeros((len(wtest), n))
            t = np.zeros((len(wtest), n, n), dtype=complex)
            wpt = np.zeros((len(wtest), n, n), dtype=complex)
            for i in range(len(wtest)):
                k2[i], t[i] = np.linalg.eigh(wtest[i])
                wpt[i] = np.matrix(t[i]).H @ (np.matrix(wptest[i]) @ np.matrix(t[i]))
            p = np.array([np.matrix(t[i]).H @ np.matrix(t[i-1]) if i > 0 else np.matrix(t[0]) for i in range(len(wtest))])
            c1 = np.zeros((len(wtest), n, n), dtype=complex)
            c2 = np.zeros((len(wtest), n, n), dtype=complex)
            c3 = np.zeros((len(wtest), n, n), dtype=complex)
            c4 = np.zeros((len(wtest), n, n), dtype=complex)
            for i in range(len(wtest)):
                alpha = np.array([-(wpt[i,j,j])**(1./3.) if np.allclose(-(wpt[i,j,j])**(1./3.), np.real(-(wpt[i,j,j])**(1./3.)))
                                  else -(wpt[i,j,j])**(1./3.)*np.exp(-1j*np.angle(-(wpt[i,j,j])**(1./3.))) for j in range(n)])
                beta = k2[i]/np.diag(wpt[i])
                wr = alpha/np.pi
                rhob = np.diff(r)[i]/2
                x = np.array([[alpha[j]*(-rhob+beta[j]), alpha[j]*(rhob+beta[j])] for j in range(n)])
                for j in range(n):
                    if np.any(np.real(x[j]) > 100):
                        dksi = np.diff((2./3)*(x[j]**(3./2)))[0]
                        aie, aiep, bie, biep = airye(x[j])
                        c1[i,j,j] = np.pi*(np.exp(-dksi)*aie[1]*biep[0]-np.exp(dksi)*bie[1]*aiep[0]) #/wr[j]
                        c2[i,j,j] = np.pi*(np.exp(dksi)*bie[1]*aie[0]-np.exp(-dksi)*aie[1]*bie[0])/alpha[j]
                        c3[i,j,j] = np.pi*alpha[j]*(np.exp(-dksi)*aiep[1]*biep[0]-np.exp(dksi)*biep[1]*aiep[0]) #/wr[j]
                        c4[i,j,j] = np.pi*(np.exp(dksi)*biep[1]*aie[0]-np.exp(-dksi)*aiep[1]*bie[0]) #/wr[j]
                    else:
                        ai, aip, bi, bip = airy(x[j])
                        c1[i,j,j] = np.pi*(ai[1]*bip[0]-bi[1]*aip[0]) #/wr[j]
                        c2[i,j,j] = np.pi*(bi[1]*ai[0]-ai[1]*bi[0])/alpha[j]
                        c3[i,j,j] = np.pi*alpha[j]*(aip[1]*bip[0]-bip[1]*aip[0]) #/wr[j]
                        c4[i,j,j] = np.pi*(bip[1]*ai[0]-aip[1]*bi[0]) #/wr[j]
            d1 = np.array([np.matrix(c2[i]).I @ c1[i] for i in range(len(wtest))])
            d2 = np.array([np.matrix(c2[i]).I for i in range(len(wtest))])
            d3 = np.array([c4[i] @ np.matrix(c2[i]).I @ c1[i] - c3[i] for i in range(len(wtest))])
            d4 = np.array([c4[i] @ np.matrix(c2[i]).I for i in range(len(wtest))])
            y = np.zeros((len(r), n, n), dtype=complex)
            y[0] = 1e308*np.eye(n, dtype=complex)
            for i in range(len(wtest)):
                prey = d4[i] - d3[i] @ np.matrix(y[i]+d1[i]).I @ d2[i]
                y[i+1] = np.matrix(p[i]) @ (prey @ np.matrix(p[i]).H)
            yf[l] = np.matrix(t[-1]).H @ (y[-1] @ np.matrix(t[-1]))
        ytot = np.array([(yf[l]+np.matrix(yf[l]).H)/2 for l in range(lmax)])
        jmat = np.zeros((lmax, n, n))
        nmat = np.zeros((lmax, n, n))
        jpmat = np.zeros((lmax, n, n))
        npmat = np.zeros((lmax, n, n))
        for j in range(n):
            if j <= nt:
                prej, prejp = riccati_jn(lmax-1, np.real(ks[j])*r[-1])
                pren, prenp = riccati_yn(lmax-1, np.real(ks[j])*r[-1])
                jmat[:,j,j] = prej/np.sqrt(np.real(ks[j]))
                nmat[:,j,j] = pren/np.sqrt(np.real(ks[j]))
                jpmat[:,j,j] = np.sqrt(np.real(ks[j])) * prejp
                npmat[:,j,j] = np.sqrt(np.real(ks[j])) * prenp
            else:
                jmat[:,j,j], prejp = riccati_in(lmax-1, np.real(ks[j])*r[-1])
                nmat[:,j,j], prenp = riccati_kn(lmax-1, np.real(ks[j])*r[-1])
                jpmat[:,j,j] = ks[j] * prejp
                npmat[:,j,j] = ks[j] * prenp
        rmat = np.array([-np.matrix(npmat[l] - ytot[l] @ nmat[l]).I @ np.matrix(jpmat[l] - ytot[l] @ jmat[l]) for l in range(lmax)])
        smat = np.array([np.matrix(np.eye(nt) + 1j*rmat[l,:nt,:nt]).I @ np.matrix(np.eye(nt) - 1j*rmat[l,:nt,:nt]) if not np.all(np.isnan(rmat[l,:nt,:nt])) else np.eye(nt) for l in range(lmax)])
        sigmas[ei,:nt,:nt] = np.pi * 1e-16 * np.sum(np.array([(2*l+1)*(np.abs(smat[l]-np.eye(nt))**2) for l in range(lmax)]),axis=0) @ np.matrix(np.abs(np.diag(ks[:nt]))**2).I
        print(ecm[ei])
        print(sigmas[ei])
    headerfinal = 'E, cm-1\t'
    for i in range(n):
        for j in range(n):
            headerfinal += f'Sigma_{i}{j}, cm2\t'
    tofile = np.zeros((len(es), n*n+1))
    tofile[:,0] = ecm
    for i in range(n):
        for j in range(n):
            tofile[:,i*n+j+1] = sigmas[:,i,j]

    np.savetxt(f'{dir}/sigmas_total_airy.txt', tofile, fmt='%.6e', delimiter='\t', header=headerfinal, comments='')
    print(f'Directory {dir} is done!')

H_2_p
105231.727
[[4.6992116e-14 0.0000000e+00 0.0000000e+00 0.0000000e+00]
 [0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00]
 [0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00]
 [0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00]]
105236.364
[[4.88799711e-14 0.00000000e+00 0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]]
105241.00099999999
[[5.74598567e-14 0.00000000e+00 0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]]
105245.63799999999
[[4.0710749e-14 0.0000000e+00 0.0000000e+00 0.0000000e+00]
 [0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00]
 [0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00]
 [0.0000000e+0